In [2]:
import json
import pymysql as mc
import sqlalchemy
from sqlalchemy import create_engine
from getpass import getpass
import Credentials as cred
import pandas as pd
from pandas import pivot_table
import numpy as np
from numpy.linalg import svd
import scipy
import scipy.sparse
import matplotlib.pyplot as plt 
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, KFold,PredefinedKFold, GridSearchCV
%matplotlib inline

In [3]:
pw = getpass()

········


In [6]:
cnx = create_engine('mysql+pymysql://'+cred.user+':'+pw+'@'+cred.host+':3306/'+cred.db_name,echo=False)

In [7]:
# get the users, business, and rankings for all the data
review_df = pd.read_sql_query("SELECT user_id, business_id, stars FROM review",cnx)
view_df = pd.read_sql_query("SELECT * FROM tempView2",cnx)
df = review_df.merge(view_df,on='business_id')

In [63]:
# now limit the dataset to only frequent users
lower_bound = 501
upper_bound = 5000
user_review_df = df.groupby(by='user_id').count()
frequent_posters = user_review_df[(user_review_df['business_id']>=lower_bound)&
                                  (user_review_df['business_id']<=upper_bound)                                  
                                 ].index.values
condensed_df = df[df['user_id'].isin(frequent_posters)]

In [64]:
# print out summary stats
print (f'Original distinct users was {df["user_id"].nunique()}, frequent posters make up {condensed_df["user_id"].nunique()}')
print (f'Original number restaurants was {df["business_id"].nunique()}, frequent posters make up {condensed_df["business_id"].nunique()} restaurants')

Original distinct users was 1031840, frequent posters make up 29
Original number restaurants was 57173, frequent posters make up 12725 restaurants


In [65]:
condensed_df = condensed_df.drop(columns=['categories'])
# load dataset into surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(condensed_df,reader)

In [66]:
# perform k-fold validation
kf = KFold(n_splits=5)
algo = SVD()
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)    
    accuracy.rmse(predictions,verbose=True)

RMSE: 0.8365
RMSE: 0.8293
RMSE: 0.8493
RMSE: 0.8579
RMSE: 0.8447


In [70]:
len(predictions)

4145

In [71]:
len(testset)

4145

In [73]:
type(trainset)

surprise.trainset.Trainset

In [94]:
x = zip(*predictions)
x = [w for w in x]

In [104]:
user_list, restaurant_list, actual_rating_list, expected_rating_list,impossible_list = x

In [106]:
user_list[0]

'kjeX2RXvW7RhBbD2QLd5jA'

In [109]:
oneguy = pd.read_sql_query(f"SELECT business_id, stars FROM Review WHERE user_id='{user_list[0]}'",cnx)
oneguy.head()

,business_id,stars
0,oxv2UyI5yF6yawPU20tOxA,3
1,TubbskG3TSU9TpgFFH2How,5
2,VLDFjeqpUgWhnVuB_8GuEg,2
3,xIbtbAJLWLNZcqfqF4f-Nw,5
4,3MutZJ4mfM3Ds450rbpMiw,3


In [108]:
restaurants = pd.read_sql_query(
    """SELECT business.business_id, categories,
        name, stars
        FROM business",cnx)
restaurants.head()

,business_id,categories,name
0,Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Minhas Micro Brewery
1,AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",CK'S BBQ & Catering
2,O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",La Bastringue
3,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Geico Insurance
4,8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Action Engine


In [112]:
oneguy.join(restaurants,on='business_id',how='left')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat